# Regresja logistyczna, walidacja, krzywe ROC

## Część I: regresja logistyczna
Ten notebook pomoże Ci zapoznać się z regresją logistyczną. 

Zbudujemy klasyfikator bazujący na regresji logistycznej. Jego zadaniem będzie określanie prawdopodobieństwa przyjęcia kandydata na studia na podstawie wyników z dwóch egzaminów maturalnych (każdy przeskalowany na zakres 0-100%): z matematyki i z biologii. 

Trzeba będzie uzupełnić kody funkcji:
*     sigmoida
*     funkcjaKosztu
*     predykcja
*     funkcjaKosztuReg

Zanim przejdziemy do właściwych zadań zaimportujmy potrzebne moduły:

In [ ]:
import matplotlib
import numpy as np
import pylab as plt
import scipy.optimize as so
from ipywidgets import FloatProgress
from IPython.display import display

Dane do ćwiczenia zostały już opracowane przez wykładowcę i znajdują się w pliku:

https://brain.fuw.edu.pl/edu/images/d/d8/Reg_log_data1.txt.

Można je pobrać i zapisać na dysku Google'a (lub w bieżącym katalogu, jeśli pracujecie lokalnie), a następnie go podłączyć. Można załadować do bieżącej sesji Colaba. No i można pobrać z GitHuba. Dokładniejszy opis był na pierwszych zajęciach!

In [ ]:
!git clone https://github.com/Shmoo137/uczenie-maszynowe-2021-22

In [ ]:
folder = '/content/uczenie-maszynowe-2021-22/dane/' # tu się domyślnie zapisuje pobrane repo z GitHuba

##  Wczytanie danych

Zawsze dobrze jest rozpocząć pracę od zapoznania się z danymi. W tym celu powinniśmy poznać strukturę danych: 
* Pierwsze dwie kolumny zawierają wyniki egzaminów (pierwsza z matematyki, druga z biologii),
* Trzecia kolumna zawiera etykietę (przynależność do grupy), po ang. *labels*

Wczytajmy dane i zobaczmy pierwsze 10 linii.

In [ ]:
data = np.loadtxt(folder + 'reg_log_data.txt',delimiter=',')
print(data[:10,:])

Aby łatwiej było się nimi posługiwać wydzielmy z nich dane wejściowe jako 'X' i wyjściowe jako 'Y':

In [ ]:
X = 
y = 

O ile poziom skomplikowania problemu na to pozwala, zawsze dobrze jest zyskać dodatkową intuicję co do danych przez zobrazowanie ich. Zróbmy wykres punktowy (plt.scatter), w którym:
* żółty symbol oznacza przykłady gdzie y = 1 (przyjęty na studia) zaś 
* niebieski te z y = 0 (nieprzyjęty na studia)

In [ ]:
plt.figure()
plt.scatter(wynik z matematyki, wynik z biologii, c=?, cmap=plt.cm.Set3)
plt.xlabel('wynik z matematyki')
plt.ylabel('wynik z biologii')

Jak na poprzednich ćwiczeniach, wykorzystywany model ma wyraz wolny. Robimy to samo, co ostatnio, czyli aby wygodnie można było stosować oblicznia wektorowe do regresji logistycznej, to zmodyfikujemy macierz wejść X poprzez dodanie do niej z lewej strony kolumny jedynek. Będą one mnożyły  parametr $\theta_0$:

In [ ]:
N = len(y) # liczba przykładów w zbiorze uczącym
XX = np.concatenate((np.ones((N,1)), X),axis = 1)

Zobaczmy jak teraz wygląda macierz XX:    

In [ ]:
print(XX[:10,:])

## Hipoteza
Dla przypomnienia _hipoteza_ w regresji logistycznej ma postać: 

$\qquad$ $h_\theta(x) = \frac{1}{1+\exp(-\theta^Tx)}$.

W implementacji dobrze jest myśleć o tej funkcji tak:

$\qquad$ $h_\theta(x) = \frac{1}{1+f}$.

gdzie: $f = \exp(-\theta^Tx)$

* zaimplementuj  hipotezę dla regresji logistycznej, 
* ze względu na stabilność numeryczną obliczeń dobrze jest ograniczyć zakres zmienności $f$ np do zakresu [1e-8, 1e+8]:

In [ ]:
def hipoteza(x, theta):
    '''ta funkcja zwraca wartość hipotezy dla danego wejścia x i parametrów theta'''
    f = np.exp(...)
    if f < 1e-8:
        f = 1e-8
    if f > 1e8:
        f = 1e8 
    h = 1/(1+f)
    return h

Przetestuj funkcję:

In [ ]:
H0 = hipoteza(XX[0,:], np.zeros((3, 1)))
print('wartość hipotezy dla zerowego przykładu i dla początkowej thety: '+ str(H0))

Powino wyjść:

```wartość hipotezy dla zerowego przykładu i dla początkowej thety: [ 0.5]```

## Funkcja log-wiarygodności: 
* parametry regresji znajdujemy przez maksymalizację [funkcji log-wiarygodności](https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_6#Funkcja_wiarygodno.C5.9Bci):

$\qquad$ $l(\theta) = \log L(\theta) = \sum_{j=1}^m y^{(j)} \log h(x^{(j)}) + (1 - y^{(j)}) \log (1 - h(x^{(j)}))$

In [ ]:
def funkcjaLogWiarygodnosci(theta, X, y):
    '''Ta funkcja oblicza wartość funkcji log-wiarygodności  dla regresji logistycznej
    używając theta jako parametrów oraz X i y jako zbioru uczącego'''
    l=0.0
    # pętla po przykładach ze zbioru uczącego
    for j in range(len(y)): 
        h = hipoteza(...)
        l +=  ...   
    return l   

W tym ćwiczeniu zrobimy to za pomocą funkcji optymalizacyjnych z modułu [<tt>scipy.optimize</tt>]( http://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize). 

Wynikają z tego dwie konsekwencje:
* Funkcje te są przystosowane do szukania minimów funkcji celu. Musimy więc podawać im jako argumenty funkcję minus log-wiarygodności

In [ ]:
def minusFunkcjaLogWiarygodnosci(theta, X, y):
    return (-1.)*funkcjaLogWiarygodnosci(theta, X, y)

* Niektóre algorytmy mogą działać szybciej jeśli zaimplementujemy jawnie postać pochodnej:

$\qquad$ $
\begin{array}{lcl}
\frac{\partial}{\partial \theta_i} l(\theta)  =\sum_{j=1}^m (y^{(j)}-h_\theta(x^{(j)}))x_i^{(j)}
\end{array}
$

In [ ]:
def pochodnaLogWiarygodnosci(theta, X, y):
    '''ta funkcja oblicza wartość pochodnej funkcji log-wiarygodności
    dla podaanych wartości theta, X i y'''
    dl_dtheta = np.zeros(len(theta))
    for i in range(len(theta)): # dla i-tej współrzędnej theta
        for j in range(len(y)):  # dodajemy przyczynki od przykładu j-ego 
            h = ...
            dl_dtheta[i] += ...
    return dl_dtheta

In [ ]:
def minusPochodnaLogWiarygodnosci(theta, X, y):
    return (-1)*pochodnaLogWiarygodnosci(theta, X, y)

Inicjujemy parametry $\theta$ na wartości 0:

In [ ]:
xDim = XX.shape[1] # rozmiar wejścia rozszerzonego o jedynki
theta0 = np.zeros((xDim, 1));

Zobaczmy jak wygląda początkowa $\theta$:

In [ ]:
print(theta0)

Obliczamy funkcje log-wiarygodności i jej pochodną dla danych początkowych:

In [ ]:
logWiar = funkcjaLogWiarygodnosci(theta0, XX, y)
pochLogWiar = pochodnaLogWiarygodnosci(theta0, XX, y)

print( 'wartość log-wiarygodności dla początkowej thety: '+ str(logWiar))
print( 'pochodna log-wiarygodnosci dla poczatkowej thety: '+ str(pochLogWiar))

Przy prawidłowej implementacji pownino wyjść:
```
wartość log-wiarygodności dla początkowej thety: [-69.31471806]
pochodna log-wiarygodnosci dla poczatkowej thety: [   10.          1200.92165893  1126.28422055]
```

## Optymalizacja  

* Funkcje optymalizujące zaczerpniemy z modułu scipy.optimize
* Ponieważ funkcje te są zaimplementowane do mnimalizowania to zamiast maksymalizować funkcję low-wiarygodności będziemy minimalizować tą funkcje przemnożoną przez -1 czyli minusFunkcjaLogWiarygodnosci fprime=minusPochodnaLogWiarygodnosci,

In [ ]:
theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0, 
                         fprime=minusPochodnaLogWiarygodnosci, 
                         args=(XX,y), disp= True)

Wypiszmy dopasowane $\theta$:

In [ ]:
print( 'Wartość log wiarygodnosci  dla optymalnych parametrów: '+
      str(funkcjaLogWiarygodnosci(theta_opt, XX, y)))
print( 'theta: '+str(theta_opt))

## Wyniki
Wyniki regresji logistycznej możemy odbierać na dwa sposoby:
* obliczyć wartość hipotezy dla badanego wejścia i dopasowanych parametrów: miara ta ma interpretację prawdopodobieństwa przynależności wejścia do klasy 1,
* dopisać funkcję wykonującą klasyfikację, tzn. porównanie wartości hipotezy z 1/2: 
  * dla wartości hipotezy > 1/2 klasyfikacja zwraca 1, 
  * w przeciwnym razie 0.

In [ ]:
def klasyfikacja(testX, theta):
    ''' Ta funkcja zwraca wynik klasyfikacji przykładu testX przy parametrach theta.
    Po obliczeniu hipotezy, jeśli otrzymane prawdopodobieństwo jest większe niż 0.5 to 
    zwraca 1 w przeciwnym wypadku zwraca 0'''
    h = ...
    if h > ... :
        klasa = ...
    else:
        klasa = ...
        
    return klasa

## Przewidywanie 
Po dopasowaniu parametrów nadszedł czas aby zrobić predykcję.
Obliczmy jakie prawdopodobieństwo przyjęcia ma kandydat z wynikami
* 20 z matematyki
*  80 z biologii
Do przewidywania wykorzystujemy funkcję hipoteza, bo zgodnie z naszą interpretacją daje ona prawdopodobieństwo przyjęcia

In [ ]:
prob = hipoteza([1, 20, 80], theta_opt)
print('dla kandydata z wymnikami 20 z matematyki i 80 z biologii prawdopodobieństwo przyjęcia wynosi: ' +str(prob))

Używając funkcji <tt>klasyfikacja</tt> dostaniemy klasę:
    

In [ ]:
klasa = klasyfikacja([1, 20, 80], theta_opt)
print('kandydat zalicza się do klasy: ' +str(klasa))

Narysujmy uzyskany podział. Na tle punktów pokolorowanych zgodnie z przynalenością do klas dorysujemy prostą rozgraniczającą obszary "1" od "0".   Ma ona równanie 

$\qquad$ $h_\theta(x)=1/2$, 

tzn:

$\qquad$ $\theta^T x = 0$

czyli 

$\theta_0 +\theta_1 x_1 + \theta_2 x_2 =0 $

Przekształcając to do równania prostej we współrzędnych $(x_1,x_2)$ mamy:

$- \theta_2 x_2 = \theta_0 +\theta_1 x_1 $

$ x_2 = - \frac{1}{\theta_2}( \theta_0 +\theta_1 x_1 )$

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set3)
plt.xlabel('wynik z matematyki')
plt.ylabel('wynik z biologii')

x_plot = np.array([np.min(X[:,1]), np.max(X[:,1])])
y_plot = -1./theta_opt[2]*(theta_opt[1]*x_plot + theta_opt[0])

plt.plot(x_plot,y_plot,'b')

## Walidacja
Teoria do tej części znajduje się tu:

https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_Ocena_jakości_klasyfikacji

### Zastosowanie w naszym przykładzie
Dodamy teraz kross-walidację typu leave-one-out.
Po kolei odłożymy po jednym przykładzie ze zbioru uczącego i na takim zredukownaym zbiorze nauczymy regresję, a następnie sprawdzimy która z poniższych możliwych sytuacji zachodzi:
* TP:  stan faktyczny jest pozytywny (y=1) i klasyfikator się nie myli (wynik = 1)
* TN:  stan faktyczny jest negatywny (y=0) i klasyfikator się nie myli (wynik = 0) 
* FP:   wynik fałszywie pozytywny (fałszywy alarm): stan faktyczny jest negatywny (y=0) ale klasyfikator się  myli (wynik = 1)
* FN: przegapiony alarm: stan faktyczny jest pozytywny (y=1) i klasyfikator się myli (wynik = 0)


Przygotowujemy liczniki:

In [ ]:
TP = 0 
TN = 0
FP = 0 
FN = 0

 W pętli odkładamy przykład `v` do testowania:

In [ ]:
for v in range(len(y)):
    testX = XX[v]
    testY = y[v]
    # robimy zredukowany zbiór uczący przez usunięcie przykładu v
    uczX = np.delete(XX,v,axis=0)
    uczY = np.delete(y,v)
    # uczymy regresję na zredukowanym zbiorze uczącym
    theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0,
                             fprime=minusPochodnaLogWiarygodnosci, 
                             args=(..., ...), disp= False)
   # klasyfikujemy odłożony przykład : proszę uzupełnić funkcję klasyfikacja na początku pliku
    wynik = klasyfikacja(... , ...)
    # aktualizujemy liczniki; proszę uzupełnić kod:
    if testY == 1 and wynik == 1:
            ... += 1
    if testY == 1 and wynik == 0:
            ... +=1           
    if testY == 0 and wynik == 1:
            ... +=1
    if testY == 0 and wynik == 0:
            ... +=1
print('TP: ', TP)
print('FP: ', FP)
print('TN: ', TN)
print('FN: ', FN)

Dla naszego zbioru uczącego powinniśmy uzyskać:
```
TP:  55
FP:  6
TN:  34
FN:  5
```

## Krzywa ROC

Aby wykreślić krzywą ROC należy przeprowadzić klasyfikację dla wielu możliwych wartości progu dla hipotezy, powyżej którego uznajemy przypadek za należący do klasy 1.


Modyfikujemy funkcję klasyfikacja, tak aby wynik zależał też od progu:

In [ ]:
def klasyfikacjaProg(testX, theta,prog):
    h = hipoteza(testX, theta)
    if h > prog:
        klasa = ...
    else:
        klasa = ...
    return klasa

Funkcję tą możemy wykorzystać do obliczenia liczebności poszczególnych przypadków klasyfikacji w zależności od progu:

In [ ]:
def liczROC(XX,y,progi):
    '''funkcja oblicza FPR i TPR dla zadanych progów,
       progi dla których mają być wyliczone wyniki podajemy w postaci wektora
       '''
    TP = np.zeros(len(progi))
    TN = np.zeros(len(progi))
    FP = np.zeros(len(progi))
    FN = np.zeros(len(progi))
    f = FloatProgress(min=0, max=len(y))
    display(f)
    for v in range(len(y)):
        f.value+=1
        
        testX = XX[v]
        testY = y[v]
        tenX = np.delete(XX,v,axis=0)
        tenY = np.delete(y,v)
        theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0, 
                                 fprime=minusPochodnaLogWiarygodnosci, 
                                 args=(tenX,tenY), disp= False)
        for ind, prog in enumerate(progi):
            wynik = klasyfikacjaProg(testX, theta_opt,prog)
           #==========================
           #      tu wstaw odpowiedni kawałek kodu
           #==========================
            ...
            
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN) 
    return (FPR,TPR)


Do wykreślenia krzywej ROC możesz użyć następującego kodu. Zaznaczamy w nim na wykresie wartości progów dla których osiągnięto konkretne wartości FPR i TPR.

In [ ]:
progi = np.arange(0.0,1.1,0.1)
FPR,TPR= liczROC(XX,y,progi)
plt.figure()
plt.plot(FPR,TPR,'o')
plt.plot(FPR,TPR)
for ind,pr in enumerate(progi):
    plt.text(FPR[ind],TPR[ind],str(pr))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.xlim((0,1))
plt.ylim((0,1))
plt.show()

Policzmy jeszcze pole pod krzywą ROC, tzw. AUC (area under curve), za pomocą metody trapezów: 

In [ ]:
AUC = 0
for ind in range(len(progi)-1):
    a = ...
    b = ...
    h = ...
    AUC += 0.5*(a+b)*h
print('AUC: ',AUC)

Spodziewamy się wyniku:
```
AUC:  0.9375
```